In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

result = []
driver = webdriver.Chrome()  # 또는 webdriver.Chrome(executable_path=...)

total_pages = 277

for pg in range(1, total_pages+1):
    url = f"https://portal.kfb.or.kr/consumer/branch.php?pg={pg}&idx=&col=&sw=&gubun=&orderby=&code=&data_year=&s_que1=&s_que2=&s_que3="
    driver.get(url)
    time.sleep(1.5)  # 로딩 대기
    rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")[1:]  # 헤더 제외
    for row in rows:
        tds = row.find_elements(By.TAG_NAME, "td")
        if len(tds) != 4:
            continue
        은행 = tds[0].text.strip()
        지점 = tds[1].text.strip()
        주소 = tds[2].text.strip()
        result.append({"은행": 은행, "지점": 지점, "주소": 주소})

driver.quit()
df = pd.DataFrame(result)
print(df)


           은행          지점                           주소
0     BNK경남은행          LH  경상남도 진주시 충무공동 충의로 19 LH진주사옥
1     BNK경남은행       가산디지털       서울특별시 금천구 가산동 가산디지털 1로
2     BNK경남은행     가음정금융센터          경상남도 창원시 성산구 대정로 36
3     BNK경남은행          강남       서울특별시 강남구 대치동 영동대로 421
4     BNK경남은행        거제고현      경상남도 거제시 고현동 거제중앙로 1895
...       ...         ...                          ...
5529   한국씨티은행          울산              울산광역시 남구 달동 번영로
5530   한국씨티은행          제주          제주특별자치도 제주시 노형동 도령로
5531   한국씨티은행    중앙기업업무센터         서울특별시 종로구 신문로2가 새문안로
5532   한국씨티은행          청주            충청북도 청주시 서원구 1순환로
5533   한국씨티은행  커머셜기업금융1센터         서울특별시 종로구 신문로2가 새문안로

[5534 rows x 3 columns]


In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME") 
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [ ]:
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd
db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

In [2]:
df.to_sql(name='bank_address', con=db_connection, if_exists='append',index=False)

NameError: name 'df' is not defined

In [3]:
import pandas as pd

df = pd.read_csv("bank_address.csv")
df.head()

C:\Users\ghtjq\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,은행,지점,주소
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895


In [6]:
####### 도로명주소 위도 경도 값으로 바꿔주기 ########
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [ ]:
#####주소를 위,경도 값으로 변환하기 #####
df.위도, df.경도 = df.apply(lambda x: geocoding(x.주소), axis = 1)
df.head()